## Watusi Legal Insight

### Your FIrst STop for Legal Clarity

In [2]:
#!pip install chromadb
# !pip install ragas


from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
import os
import regex as re
from datetime import datetime
from typing import TypedDict, List, Dict, Annotated
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import NotRequired
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.graph import StateGraph, START, END
from langchain.schema import Document
from langchain_core.messages import AIMessage,HumanMessage, SystemMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder)
from langchain.tools.retriever import create_retriever_tool
from langchain_ollama import OllamaLLM
from langchain_openai import ChatOpenAI
import json
from langchain_chroma import Chroma
import chromadb
from langchain_mistralai import MistralAIEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import JinaRerank
from mistralai import Mistral
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_groq.chat_models import ChatGroq
import uuid

import warnings
warnings.filterwarnings('ignore')

## Load Documents and Convert to String

In [3]:
folder_path = 'documents'

noise_patterns = [
    r'www\.hukumonline\.com',
    r'\nMenemukan kesalahan ketik dalam dokumen\?\nKlik di sini\nuntuk perbaikan.'
]
# pattern = r'(www\.hukumonline\.com|\nMenemukan kesalahan ketik dalam dokumen\?\nKlik di sini\nuntuk perbaikan.)'
all_docs = []

for file in os.listdir(folder_path):
    if file.endswith('.pdf'):
        file_path = os.path.join(folder_path, file)
        loader = PyPDFLoader(file_path)
        docs = loader.load()
        for doc in docs:
            for noise in noise_patterns:
                doc.page_content = re.sub(noise, '', doc.page_content)
        
        all_docs.extend(docs)

full_text = ''.join(page.page_content for page in all_docs)

## Parse The Document into Customize Documents

In [4]:
document_title_pattern = r'kitab\s+undang-undang\s+hukum\s+perdata'
document_title = re.search(document_title_pattern, full_text, re.IGNORECASE) 
document_title.group(0) #document_title


#Split berdasarkan buku ex:BUKU KETIGA, BUKU KEEMPAT
list_buku = []
list_bab = []
docs = []
buku_chunks = re.split(r'(?=BUKU\s+KE\w+)', full_text)
for buku in buku_chunks:
    buku_pattern = r'(BUKU\s+KE\w+).*'
    buku_match = re.search(buku_pattern, buku, re.DOTALL)
    # if not re.search(buku_pattern, buku, re.DOTALL):
    if not buku_match:
        continue
    list_buku.append(buku_match.group(1))
    # print(f'{buku_match.group(1)}')

    #Split berdasarkan bab
    bab_chunks = re.split(r'(?=BAB\s+[IVXLCDM]+[A-Z]?)', buku)
    for bab in bab_chunks:
        bab_pattern = r'(BAB\s+[IVXLCDM]+[A-Z]?)\s*\n(.*?)\s*\n(.*)'
        # bab_pattern = r'BAB\s+([IVXLCDM]+)\s*\n((?:.(?!\nPasal\s*\d))+)'
        bab_match = re.search(bab_pattern, bab, re.DOTALL)
        if not bab_match:
            continue
        bab_number = bab_match.group(1).strip()
        bab_title = bab_match.group(2).strip()
        bab_title = re.sub(r'\s*\n\s*',' ', bab_title)
        # print(f'{buku_match.group(1)} -> {bab_match.group(1)}: {bab_match.group(2)}')

        #Proses pasal yang ada dalam 'BAGIAN' di setiap BABnya
        if 'BAGIAN' in bab:
            bagian_chunks = re.split(r'(?=BAGIAN\s+[0-9]+)', bab)
            for bagian in bagian_chunks:
                bagian_pattern = r'(BAGIAN\s+[0-9]+)\s*\n*(.*?)\s*\n\s*\n(.*)'
                bagian_match = re.search(bagian_pattern, bagian, re.DOTALL)
                if not bagian_match:
                    continue
                bagian_number = bagian_match.group(1).strip()
                bagian_title=bagian_match.group(2).strip()
                # print(f'{buku_match.group(1)} -> {bab_number}: {bab_title} -> {bagian_number}: {bagian_title}')

                # Proses Pasal
                pasal_chunks = re.split(r'(?=Pasal\s+[0-9]+[a-z]?)', bagian, re.DOTALL)
                for pasal in pasal_chunks:
                    pasal_pattern = r'(Pasal\s+[0-9]+[a-z]?)\s*\n(.*)'
                    pasal_match = re.search(pasal_pattern, pasal, re.DOTALL)
                    if not pasal_match:
                        continue
                    pasal_number = pasal_match.group(1).strip()
                    pasal_content = pasal_match.group(2).strip()
                
                    doc = Document(
                        page_content =
                            f"Dokumen: {document_title.group(0)}\n"
                            f"Bab: {bab_number} - {bab_title}\n"
                            f"Bagian: {bagian_number} - {bagian_title}\n"
                            f"Pasal: {pasal_number}\n"
                            f"Isi Pasal: {pasal_content}",
                        metadata = {
                        'document_type': 'legal_document',
                        'document_title': document_title.group(0) + ' ' + buku_match.group(1),
                        'bab_number': bab_number,
                        'bab_title': bab_title,
                        'bagian': bagian_number,
                        'bagian_title': bagian_title,
                        'pasal': pasal_number}
                    )
                    docs.append(doc)

        # Proses Pasal Tanpa Bagian
        else:
            pasal_chunks = re.split(r'(?=Pasal\s+[0-9]+[a-z]?)', bab, re.DOTALL)
            for pasal in pasal_chunks:
                    pasal_pattern = r'(Pasal\s+[0-9]+[a-z]?)\s*\n(.*)'
                    pasal_match = re.search(pasal_pattern, pasal, re.DOTALL)
                    if not pasal_match:
                        continue
                    pasal_number = pasal_match.group(1).strip()
                    pasal_content = pasal_match.group(2).strip()
                    doc = Document(
                        page_content =
                            f"Dokumen: {document_title.group(0)}\n"
                            f"Bab: {bab_number} - {bab_title}\n"
                            f"Bagian: {bagian_number} - {bagian_title}\n"
                            f"Pasal: {pasal_number}\n"
                            f"Isi Pasal: {pasal_content}",
                        metadata = {
                        'document_type': 'legal_document',
                        'document_title': document_title.group(0) + ' ' + buku_match.group(1),
                        'bab_number': bab_number,
                        'bab_title': bab_title,
                        'bagian': '',
                        'bagian_title': '',
                        'pasal': pasal_number}
                    )
                    docs.append(doc)

## RAG Pipeline

### Embed Documents, Store in Vector DB and Rerank with Cross-Encoder

In [5]:
#load api key
with open('credentials.json', 'r')as f:
    data = json.load(f)

mistral_api = data['mistral_api']
jina_api = data['jina_api']
#------------------------------------------------------------------------------------------------------------------------------------------------------------

#load embedding model
model_name = 'mistral-embed'
embedding_model = MistralAIEmbeddings(
    model = model_name,
    api_key=mistral_api)
#------------------------------------------------------------------------------------------------------------------------------------------------------------

#store / load document from chroma db
vector_store_mistral = Chroma(
    embedding_function=embedding_model,
    persist_directory='.\pipeline\chroma_db',
    collection_name = 'kuhperdata_mistral'   
)
# vector_store.add_documents(docs) -> Run this script to add document to vector db
#------------------------------------------------------------------------------------------------------------------------------------------------------------

#set vector db as retriver with search type similarity and get 'top 5' documents
retriever = vector_store_mistral.as_retriever(
    search_type = 'similarity',
    k= 5)

#rerank the documents with the help of Cross Encoder model (in my case I'm utilazing Jina Reranker model)
compressor = JinaRerank(jina_api_key = jina_api)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever)
#------------------------------------------------------------------------------------------------------------------------------------------------------------

# Run code below to identified what vector db (collection) that available in the local storage
# client = chromadb.PersistentClient(path = './pipeline/chroma_db/')
# client.list_collections()

In [6]:
def debug_retrieval(retriever, query, expected_content=None):
    """Debug retrieval dengan eye-balling results"""
    results = retriever.similarity_search_with_score(query, k=3)
    
    print(f"Query: {query}")
    print("=" * 50)
    
    for i, (doc, score) in enumerate(results):
        print(f"Rank {i+1} | Score: {score:.4f}")
        print(f"Content: {doc.page_content[:200]}...")
        if expected_content and expected_content.lower() in doc.page_content.lower():
            print("✅ CONTAINS EXPECTED CONTENT!")
        print("-" * 30)

# Test
# debug_retrieval(vector_store_mistral, "Pasal 1234", "Perikatan ditujukan untuk memberikan sesuatu, untuk berbuat sesuatu, atau untuk tidak berbuat sesuatu")

### Load LLM Model

1. Mistral LLM for RAG Chain -> *gemma model as alternative with local environment*
2. Gemini LLM for RAG Evaluation

In [7]:
with open('credentials.json', 'r')as f:
    data = json.load(f)

mistral_api = data['mistral_api']
open_route_api = data['open_route_api']
groq_api = data['groq_api']

#load gemma model
ollama_llm = OllamaLLM(
    model = 'gemma3:1b',
    temperature = 0.2)

#load mistral model
mistral_llm = ChatMistralAI(
    # model = 'Hermes-2-Pro-Mistral-7B',
    model = 'mistral-small-2506',
    api_key=mistral_api,
    temperature = 0.1
)

#load gemini model
openrouter_llm = ChatOpenAI(
    model = 'google/gemini-2.0-flash-exp:free',
    base_url="https://openrouter.ai/api/v1",
    api_key = open_route_api
)

### Create RAG Chain Utilizing Langchain and Langgraph

In [8]:
class OverallState(TypedDict):
    input: str
    retrieved_docs: List[Document]
    prompt: str
    messages: Annotated[List[BaseMessage], add_messages]
    llm: object
    nama: NotRequired[str]
    domisili: NotRequired[str]


def retriever_node(state:OverallState) -> dict:
    input = state['input']
    # docs = retriever.get_relevant_documents(input)
    docs = [doc for doc in compression_retriever.get_relevant_documents(input)]

    return({'input':input, 'retrieved_docs':docs})

def merger_node(state:OverallState) -> dict:
    # user_input = state['input']
    context = '\n'.join([doc.page_content for doc in state['retrieved_docs']])
    system_prompt = f"""
    Kamu adalah seorang ahli hukum. Jawab pertanyaan user berdasarkan Konteks: 
    {context}

    ### Instruksi:
    - Jawab selalu dalam **bahasa Indonesia** dan **output harus di bawah 50 kata**
    - **Jawaban langsung ke inti**
    - Jika pertanyaannya meminta isi pasal (misalnya menyebut "Pasal 1320", "bunyi Pasal", dsb), kutip isi pasal yang diminta **secara langsung** dari konteks.
    - Jika pertanyaannya berupa kasus hukum (contoh: pelanggaran perjanjian kerja sama, wanprestasi, dll), berikan pasal **yang relevan saja** dari konteks, jangan mengarang.
    - Jika informasi tidak ditemukan dalam konteks, katakan bahwa **informasi tidak tersedia** atau **ajukan pertanyaan untuk meminta kejelasan ke user**
    
    ### Deteksi Lanjutan (Sinyal Eskalasi):
    Jika user menunjukkan salah satu dari hal berikut, barulah berikan **kontak pengacara**:
    1. Menyatakan **bingung secara hukum**, misal: “Saya tidak tahu harus bagaimana”, “Tolong bantu saya”
    2. Menyatakan ingin **dibantu secara langsung**
    3. Menyebut butuh bantuan **membuat/review dokumen**
    4. Menunjukkan **keputusasaan/kebingungan yang jelas**
    5. Menyatakan ingin **berkonsultasi lebih lanjut**
    6. Jangan berikan tawaran bantuan hukum **di awal percakapan**

    ### Jika Sinyal Terdeteksi:
    Tambahkan di akhir jawaban:
    "Jika Anda membutuhkan bantuan lebih lanjut, silakan hubungi kami melalui email [Watusi Legal Insight](contact@watusilegalinsight.com) atau WhatsApp 0812-3456-7890. Tim kami akan menghubungi Anda setelah verifikasi data dilakukan."

    **Jangan berikan kontak jika belum ada sinyal di atas.**
    """

    prompt = ChatPromptTemplate([
        ('system', system_prompt),
        MessagesPlaceholder(variable_name = 'messages'),
        ('human', state['input'])
        ])
    # print(prompt)
    return({
        'prompt':prompt,
        'messages':[HumanMessage(content = state['input'])]})

def output_node(state:OverallState) -> dict:
    qa_chain = state['prompt'] | state['llm']
    result = qa_chain.invoke({'messages': state['messages']})
    new_messasges = state['messages'] + [result]
    # print(f"[DEBUG] Model result: {result}")
    return ({'messages': new_messasges})

# def summary_node (state: OverallState) -> dict:
    

builder = StateGraph(OverallState)
builder.add_node('retriever', retriever_node)
builder.add_node('merger', merger_node)
builder.add_node('output', output_node)
builder.add_edge(START, 'retriever')
builder.add_edge('retriever', 'merger')
builder.add_edge('merger', 'output')
builder.add_edge('output', END)

checkpointer = InMemorySaver()
store = InMemoryStore()
graph_memory = builder.compile(checkpointer=checkpointer, store = store)
graph = builder.compile()
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}    


### Chatbot Function to Retrieve Question (chat bot with memory and not)

In [9]:
def chat_bot_with_memory(question:str, llm = None):
    if llm is None:
        llm = mistral_llm
        final_result = None

        for chunk in graph_memory.stream({'input': question, 'llm': llm}, config, stream_mode = 'values'):
            final_result = chunk
        for i in range(len(final_result['messages'])):
            final_result['messages'][i].pretty_print()
        return final_result

def chat_bot(question: str, llm = None):
    if llm is None:
        llm = mistral_llm
    result = graph.invoke({'input':question, 'llm':llm})
    return result['messages'][-1].content

In [ ]:
# question = 'bunyi pasal 1234'
# result = graph.invoke({'input':question})

In [11]:
question = 'Kantor saya tidak membayarkan upah selama 3 bulan terakhir, bisa dikenakan pasal berapa aja ?'
final_result = chat_bot_with_memory(question)

================================ Human Message =================================

Kantor saya tidak membayarkan upah selama 3 bulan terakhir, bisa dikenakan pasal berapa aja ?
================================== Ai Message ==================================

Pasal 1602q KUHPer: Jika upah tidak dibayar paling lambat hari kerja ketiga setelah hari pembayaran, buruh berhak atas tambahan upah 5% per hari hingga hari ke-8 dan 1% per hari setelahnya, maksimal 50% dari jumlah upah.


### Function to Summarize Conversation (Based on Chatbot with Memory)

dict

In [20]:
def conv_summarizer(conversation:dict):
    conversation = ""
    for msg in final_result['messages']:
        if isinstance(msg, HumanMessage):
            role = 'user'
        else:
            role = 'ai'

        conversation += f"{role}: {msg.content}\n\n"

    system_prompt ="""Buat ringkasan percakapan ini maksimal 100 kata. 
            Jelaskan inti permasalahan user, apa jawaban dari AI, dan apakah ada indikasi bahwa user butuh bantuan hukum lanjutan atau tidak to the point langsung ringkas tanpa header apapun."""
    prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            ("human", conversation)])

    qa_chain = prompt | mistral_llm

    chat_sumamry = qa_chain.invoke({}).content
    return chat_sumamry

conv_summarizer(final_result)

'User tidak dibayar upah 3 bulan, AI menjelaskan pasal 1602q KUHPer tentang tambahan upah jika upah tidak dibayar tepat waktu. Tidak ada indikasi user butuh bantuan hukum lanjutan.'

## Rag Evaluation with RAGAS Framework

In [22]:
import random
import pandas as pd
from datasets import Dataset

docs_inference = random.sample(docs, 20)

questions = []
ground_truths = []

dataset = []
for doc in docs_inference:
    question = f'Bunyi {doc.metadata.get("pasal")} adalah ?'
    # ground_truth = re.sub('\nIsi Pasal:', ',', re.findall(r'(Pasal\s\d+\w*\n.*)',doc.page_content)[0])
    ground_truth = doc.page_content
    
    questions.append(question)
    ground_truths.append(ground_truth)


answers = []
contexts = []
for question in questions:
    answers.append(chat_bot(question, openrouter_llm))
    contexts.append([doc.page_content for doc in compression_retriever.get_relevant_documents(question)])

data = (
    {'user_input': questions,
    'response': answers,
    'retrieved_contexts': contexts,
    'reference': ground_truths})


RateLimitError: Error code: 429 - {'error': {'message': 'Provider returned error', 'code': 429, 'metadata': {'raw': 'google/gemini-2.0-flash-exp:free is temporarily rate-limited upstream. Please retry shortly, or add your own key to accumulate your rate limits: https://openrouter.ai/settings/integrations', 'provider_name': 'Google'}}, 'user_id': 'user_30JgS4W6yJ9QzMABCrko9J1YoDx'}

In [ ]:
dataset = Dataset.from_dict(data)

In [ ]:
from ragas import evaluate
from ragas.metrics import (faithfulness, answer_relevancy, context_recall, context_precision)
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(mistral_llm)
result = evaluate(dataset=dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)

Evaluating: 100%|██████████| 15/15 [00:27<00:00,  1.84s/it]


{'context_recall': 1.0000, 'faithfulness': 0.9000, 'factual_correctness(mode=f1)': 0.5480}

In [ ]:
print(result)

{'context_recall': 1.0000, 'faithfulness': 0.9000, 'factual_correctness(mode=f1)': 0.5480}
